# Web Scraping
Según el procedimiento en https://www.datacamp.com/community/tutorials/web-scraping-using-python

In [1]:
#Importes necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
#Establecer URL para scraping
url = "https://www.elsiglodetorreon.com.mx/archivo/?ultimas"
html = urlopen(url)

In [3]:
#Establece variable para usar Beautiful Soup
soup = BeautifulSoup(html, 'lxml')

bs4.BeautifulSoup

In [10]:
# (Prueba) Imprime título de la página
title = soup.title
##print(title)

In [11]:
# (Prueba) Imprime el texto de la página completa
text = soup.get_text()
##print(soup.text)

In [6]:
#Establece la fecha del día de hoy en una variable
from datetime import date

today = date.today()
hoy = today.strftime("%Y-%m-%d")

In [7]:
#Encuentra cada titular ( <li> en la <ul> con clase "list-group"), extrae el texto, 
#lo escribe en un archivo con nombre de fecha de hoy, y añade una segunda columna con la fecha. 
for ultag in soup.find_all('ul', {'class': 'list-group'}):
...     for litag in ultag.find_all('a'):
...             f = open('%s.csv' % hoy, "a", encoding='utf-8')
...             f.write("'")
...             f.write(litag.text)
...             f.write("', ")
...             f.write(hoy)
...             f.write("\n")
...             f.close()


### Pendientes: 
- Cuando el titular tiene una coma, produce errores en el CSV  
- Crear dataframe para su uso con Matplotlib  
- Añadir lista de HTML para ingreso a cada una de las notas  
- Como añadir varias páginas  